In [11]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import csv
import scipy as sp
import scipy.sparse as sps
from scipy.sparse import coo_matrix
import os
import matplotlib.pyplot as pyplot
%matplotlib inline 


## Evaluation or scoring?
eval = True

In [12]:
from RecLib.DataLoad import *
# Loading from CSV files...
UCM_age = toCoo('data_UCM_age.csv', 'user', 'age')
ICM_subclass = toCoo('data_ICM_sub_class.csv', 'item', 'subclass')
ICM_asset = toCoo('data_ICM_asset.csv', 'item', 'asset')
ICM_price = toCoo('data_ICM_price.csv', 'item', 'price')
UCM_region = toCoo('data_UCM_region.csv', 'user', 'region')
target_users = toNPArray('data_target_users_test.csv')
URM = toCoo('data_train.csv', 'user', 'item')

data_UCM_age.csv
user data, ID: min 2, max 30909, length 30317, unique 30317, repetitions 0, missig 1.91 %
age data, ID: min 1, max 10, length 30317, unique 10, repetitions 30307, missig 0.00 %

data_ICM_sub_class.csv
item data, ID: min 0, max 18494, length 18495, unique 18495, repetitions 0, missig 0.00 %
subclass data, ID: min 1, max 2010, length 18495, unique 1905, repetitions 16590, missig 5.18 %

data_ICM_asset.csv
item data, ID: min 0, max 18494, length 18490, unique 18490, repetitions 0, missig 0.02 %
asset data, ID: min 0, max 0, length 18490, unique 1, repetitions 18489, missig 0.00 %

data_ICM_price.csv
item data, ID: min 0, max 18494, length 18493, unique 18493, repetitions 0, missig 0.01 %
price data, ID: min 0, max 0, length 18493, unique 1, repetitions 18492, missig 0.00 %

data_UCM_region.csv
user data, ID: min 0, max 30910, length 26609, unique 26375, repetitions 234, missig 14.67 %
region data, ID: min 0, max 7, length 26609, unique 7, repetitions 26602, missig 0.00 %


In [13]:
URM = URM.tocsr()

In [14]:
## Split dataset (train % of .9999 gives similar performance on test set and competition set)
from Notebooks_utils.data_splitter import train_test_holdout
URM_train, URM_test = train_test_holdout(URM, train_perc = 0.8)


if not eval:
    URM_train = URM
else:
    from Base.Evaluation.Evaluator import EvaluatorHoldout
    evaluator_validation = EvaluatorHoldout(URM_test, cutoff_list=[10])

In [15]:
## SLIM BPR Recommender - MAP 0.41 on test set
## set True if you want to train and compile SLIM BPR Recommender
if True:
    from SLIM_BPR.Cython.SLIM_BPR_Cython import SLIM_BPR_Cython 
    slim_rec = SLIM_BPR_Cython(URM_train, recompile_cython=False, verbose = False)
    MAP_LIST = []
    epochsList = [300]
    batchSize = [50]
    tklist = [10]
    lrs = [1e-3]

    for epochsN in epochsList:
        for bs in batchSize:
            for tk in tklist:
                for lr in lrs:
                    do_not_display_hystory = slim_rec.fit(epochs=epochsN, batch_size=bs, sgd_mode='adagrad', learning_rate=lr, topK = tk)
                    if eval:
                        dict_scores = (evaluator_validation.evaluateRecommender(slim_rec))[0][10]
                        MAP_LIST.append(('epoch, batch, topK, lr :' + str(epochsN) + ' ' + str(bs)+ ' ' + str(tk) + ' ' + str(lr), dict_scores['MAP']))
    if eval:
        print(MAP_LIST)

Unable to read memory status: list index out of range
SLIM_BPR_Recommender: Epoch 1 of 300. Elapsed time 0.21 sec
SLIM_BPR_Recommender: Epoch 2 of 300. Elapsed time 0.33 sec
SLIM_BPR_Recommender: Epoch 3 of 300. Elapsed time 0.43 sec
SLIM_BPR_Recommender: Epoch 4 of 300. Elapsed time 0.51 sec
SLIM_BPR_Recommender: Epoch 5 of 300. Elapsed time 0.58 sec
SLIM_BPR_Recommender: Epoch 6 of 300. Elapsed time 0.66 sec
SLIM_BPR_Recommender: Epoch 7 of 300. Elapsed time 0.73 sec
SLIM_BPR_Recommender: Epoch 8 of 300. Elapsed time 0.80 sec
SLIM_BPR_Recommender: Epoch 9 of 300. Elapsed time 0.86 sec
SLIM_BPR_Recommender: Epoch 10 of 300. Elapsed time 0.94 sec
SLIM_BPR_Recommender: Epoch 11 of 300. Elapsed time 1.00 sec
SLIM_BPR_Recommender: Epoch 12 of 300. Elapsed time 1.07 sec
SLIM_BPR_Recommender: Epoch 13 of 300. Elapsed time 1.14 sec
SLIM_BPR_Recommender: Epoch 14 of 300. Elapsed time 1.20 sec
SLIM_BPR_Recommender: Epoch 15 of 300. Elapsed time 1.27 sec
SLIM_BPR_Recommender: Epoch 16 of 300. E

SLIM_BPR_Recommender: Epoch 135 of 300. Elapsed time 9.01 sec
SLIM_BPR_Recommender: Epoch 136 of 300. Elapsed time 9.07 sec
SLIM_BPR_Recommender: Epoch 137 of 300. Elapsed time 9.13 sec
SLIM_BPR_Recommender: Epoch 138 of 300. Elapsed time 9.19 sec
SLIM_BPR_Recommender: Epoch 139 of 300. Elapsed time 9.25 sec
SLIM_BPR_Recommender: Epoch 140 of 300. Elapsed time 9.31 sec
SLIM_BPR_Recommender: Epoch 141 of 300. Elapsed time 9.37 sec
SLIM_BPR_Recommender: Epoch 142 of 300. Elapsed time 9.43 sec
SLIM_BPR_Recommender: Epoch 143 of 300. Elapsed time 9.49 sec
SLIM_BPR_Recommender: Epoch 144 of 300. Elapsed time 9.55 sec
SLIM_BPR_Recommender: Epoch 145 of 300. Elapsed time 9.61 sec
SLIM_BPR_Recommender: Epoch 146 of 300. Elapsed time 9.67 sec
SLIM_BPR_Recommender: Epoch 147 of 300. Elapsed time 9.73 sec
SLIM_BPR_Recommender: Epoch 148 of 300. Elapsed time 9.79 sec
SLIM_BPR_Recommender: Epoch 149 of 300. Elapsed time 9.85 sec
SLIM_BPR_Recommender: Epoch 150 of 300. Elapsed time 9.91 sec
SLIM_BPR

SLIM_BPR_Recommender: Epoch 266 of 300. Elapsed time 16.95 sec
SLIM_BPR_Recommender: Epoch 267 of 300. Elapsed time 17.01 sec
SLIM_BPR_Recommender: Epoch 268 of 300. Elapsed time 17.07 sec
SLIM_BPR_Recommender: Epoch 269 of 300. Elapsed time 17.13 sec
SLIM_BPR_Recommender: Epoch 270 of 300. Elapsed time 17.19 sec
SLIM_BPR_Recommender: Epoch 271 of 300. Elapsed time 17.25 sec
SLIM_BPR_Recommender: Epoch 272 of 300. Elapsed time 17.31 sec
SLIM_BPR_Recommender: Epoch 273 of 300. Elapsed time 17.37 sec
SLIM_BPR_Recommender: Epoch 274 of 300. Elapsed time 17.43 sec
SLIM_BPR_Recommender: Epoch 275 of 300. Elapsed time 17.49 sec
SLIM_BPR_Recommender: Epoch 276 of 300. Elapsed time 17.55 sec
SLIM_BPR_Recommender: Epoch 277 of 300. Elapsed time 17.61 sec
SLIM_BPR_Recommender: Epoch 278 of 300. Elapsed time 17.68 sec
SLIM_BPR_Recommender: Epoch 279 of 300. Elapsed time 17.74 sec
SLIM_BPR_Recommender: Epoch 280 of 300. Elapsed time 17.80 sec
SLIM_BPR_Recommender: Epoch 281 of 300. Elapsed time 17

In [16]:
from RecLib.DataLoad import ICMbuilder 
ICM = ICMbuilder(URM_train, ICM_subclass, ICM_price, ICM_subclass)
ICM = sps.csr_matrix(ICM)

In [17]:
if True:
    from KNN.ItemKNNCBFRecommender import ItemKNNCBFRecommender 
    itemKNNCBF = ItemKNNCBFRecommender(URM_train, ICM)
    MAP_LIST = []
    tklist = [3]
    shrinklist = [10]

    for tk in tklist:
        for sr in shrinklist:
            itemKNNCBF.fit(shrink=sr, topK = tk, similarity = 'cosine')
            if eval:
                dict_scores = (evaluator_validation.evaluateRecommender(itemKNNCBF))[0][10]
                MAP_LIST.append(('topK, shrink :' + str(tk) + ' ' + str(sr), dict_scores['MAP']))
    if eval:
        print(MAP_LIST)



ItemKNNCBFRecommender: URM Detected 4314 (13.96 %) cold users.
ItemKNNCBFRecommender: URM Detected 3710 (20.06 %) cold items.
Similarity column 18495 ( 100 % ), 4658.37 column/sec, elapsed time 0.07 min
EvaluatorHoldout: Processed 20363 ( 100.00% ) in 10.61 sec. Users per second: 1920
[('topK, shrink :3 10', 0.010180203504862421)]


In [18]:
## ItemCFKNNRecommender - MAP 0.47 on test set
# If you want to compile and train KNN set to True
if True:
    from KNN.ItemKNNCFRecommender import ItemKNNCFRecommender
    itemKNN = ItemKNNCFRecommender(URM_train)
    itemKNN.fit(shrink=50, topK=3)

    if eval:
        results, _ = evaluator_validation.evaluateRecommender(itemKNN)
        print('MAP: ' + str(results[10]["MAP"]))

ItemKNNCFRecommender: URM Detected 4314 (13.96 %) cold users.
ItemKNNCFRecommender: URM Detected 3710 (20.06 %) cold items.
Similarity column 18495 ( 100 % ), 9630.63 column/sec, elapsed time 0.03 min
EvaluatorHoldout: Processed 20363 ( 100.00% ) in 10.27 sec. Users per second: 1982
MAP: 0.04501446583233149


In [19]:
## HYBRID: KNN + SLIM BPR
# Set to True if you want to compile and train this hybrid rec
from RecLib.HybridRecommender import *
if True: 
    hybridrecommender = HybridRecommender(URM_train, itemKNN, itemKNNCBF, slim_rec)
    hybridrecommender.fit(0.5, 0.1, 0.4)
    
    if eval:
        results, _ = evaluator_validation.evaluateRecommender(hybridrecommender)
        print('MAP: ' + str(results[10]["MAP"]))

HybridRecommender: URM Detected 4314 (13.96 %) cold users.
HybridRecommender: URM Detected 3710 (20.06 %) cold items.
TopPopRecommender: URM Detected 4314 (13.96 %) cold users.
TopPopRecommender: URM Detected 3710 (20.06 %) cold items.
ScoresRecommender: URM Detected 4314 (13.96 %) cold users.
ScoresRecommender: URM Detected 3710 (20.06 %) cold items.
(3, 1)


Exception: ERROR: parameters insufficient or wrong in number

In [ ]:
if eval:
    for itr in range(300):
        to_compute_mask = np.ediff1d(URM_train.tocsr().indptr) == itr
        to_ignore_mask = np.invert(to_compute_mask)
        to_ignore = np.arange(URM_train.shape[0])[to_ignore_mask]
        if len(to_ignore) != URM_train.shape[0]:
            evaluator_validation = EvaluatorHoldout(URM_test, cutoff_list=[10], ignore_users = to_ignore)
            results, _ = evaluator_validation.evaluateRecommender(hybridrecommender)
            print('MAP at ' + str(itr) +' interactions: ' + str(results[10]["MAP"]))

In [ ]:
## TEST & VISUALIZE DIFFERENT RECSYS PERFORMANCE BASED ON N° OF INTERACTIONS HELD BY USERS
# Please, fill empty list with pretrained recommenders you want to compare

if eval:
    recommendersToCompare = [itemKNN, hybridrecommender]
    from RecLib.PerUserGroupTesting import *
    compare(URM_train, URM_test, recommendersToCompare)

In [ ]:
## Predict
## SET WHICH RECOMMENDER YOU WANNA USE
final_Rec = hybridrecommender
if not eval:
    output = []
    for user_id in target_users:
        output.append((user_id, final_Rec.recommend(user_id, cutoff=10)))

In [ ]:
#Writedown results
if not eval:
    with open('submission.csv', 'w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(["user_id", "item_list"])
        for row in output:
          ranking = ''
          for val in row[1]:
            ranking = ranking + str(val) + ' '
          writer.writerow([row[0], ranking[:-1]])